In [1]:
import numpy as np
from math import sqrt
from torch.utils.data import DataLoader
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import os
import time
import pickle as pc
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from functools import partial
#from MFDFA import MFDFA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import matplotlib.pyplot as plt
import math
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import yfinance as yf
import datetime
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import adfuller
'''os.environ["MLFINLAB_API_KEY"] = "2f849d6ab0af7315a1de23309417fb85"
import mlfinlab
from mlfinlab.data_structures import standard_data_structures
from mlfinlab.labeling.raw_return import raw_return
from mlfinlab.sample_weights.attribution import get_weights_by_time_decay
from mlfinlab.labeling.trend_scanning import trend_scanning_labels
from sklearn.ensemble import RandomForestRegressor
from mlfinlab.util import volatility
from mlfinlab.filters import filters
from mlfinlab.feature_importance.fingerprint import RegressionModelFingerprint
from mlfinlab.labeling import labeling
from mlfinlab.networks.almst import ALMST
from mlfinlab.networks.dash_graph import DashGraph
from mlfinlab.codependence.codependence_matrix import get_distance_matrix
from mlfinlab.sample_weights.attribution import get_weights_by_return'''

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path + '/' + 'checkpoint.pth')
        self.val_loss_min = val_loss

def media_mobile(serie_dati, periodi):
    medie_mobile = []
    for i in range(len(serie_dati) - periodi + 1):
        media = sum(serie_dati[i:i+periodi]) / periodi
        medie_mobile.append(media)
    return medie_mobile

file_path = "dati.csv"
df_raw = pd.read_csv(file_path).reset_index(drop=True)
df_raw=df_raw[200:]
nomi_colonne_da_salvare = ['DATE', 'VALUE_LT_STA3']
df_raw = df_raw.loc[:, nomi_colonne_da_salvare]
df_raw.rename(columns={'DATE': 'Date'}, inplace=True)#.reset_index(drop=True)
#df_raw = df_raw.iloc[::-1].reset_index(drop=True)
#df_raw = df_raw.iloc[::6]
df_raw=df_raw[1:].reset_index(drop=True)
df_raw.to_csv('emanuele.csv', index=False)#.reset_index(drop=True)
df_raw

ModuleNotFoundError: No module named 'hyperopt'

In [2]:
df_raw[0:50]

,Date,VALUE_LT_STA3
0,2021-06-15 09:30:00,24.6
1,2021-06-15 09:40:00,24.8
2,2021-06-15 09:50:00,25.0
3,2021-06-15 10:00:00,25.1
4,2021-06-15 10:10:00,25.4
5,2021-06-15 10:20:00,25.4
6,2021-06-15 10:30:00,25.8
7,2021-06-15 10:40:00,26.1
8,2021-06-15 10:50:00,26.5
9,2021-06-15 11:00:00,26.7


In [3]:
'''correlation_matrix = df_raw.drop("DATE",axis=1).corr(method='pearson')
custom_matrix = get_distance_matrix(correlation_matrix, metric='angular')
graph = ALMST(custom_matrix, 'distance')
dash_graph = DashGraph(graph)
server = dash_graph.get_server()
server.run_server()'''

'correlation_matrix = df_raw.drop("DATE",axis=1).corr(method=\'pearson\')\ncustom_matrix = get_distance_matrix(correlation_matrix, metric=\'angular\')\ngraph = ALMST(custom_matrix, \'distance\')\ndash_graph = DashGraph(graph)\nserver = dash_graph.get_server()\nserver.run_server()'

In [4]:
'''df_raw = pd.read_csv('emanuele.csv').drop('Date', axis=1)
scaler =StandardScaler()

X = df_raw.drop('VALUE_LT_STA6', axis=1)[:-1]
X= pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

y = df_raw['VALUE_LT_STA6'].shift(1)[1:]
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg = reg.fit(X, y)
# Create the fingerprint model
reg_fingerprint = RegressionModelFingerprint()
# Fit the fingerprint model
_ = reg_fingerprint.fit(
     reg,
     X,
     num_values=20,)
# Get linear non-linear effects and pairwise effects
linear_effect, non_linear_effect, pair_wise_effect = reg_fingerprint.get_effects()
# Plot the results
fig = reg_fingerprint.plot_effects(sort_by="non_lin")
fig.show()'''

'df_raw = pd.read_csv(\'emanuele.csv\').drop(\'Date\', axis=1)\nscaler =StandardScaler()\n\nX = df_raw.drop(\'VALUE_LT_STA6\', axis=1)[:-1]\nX= pd.DataFrame(scaler.fit_transform(X), columns=X.columns)\n\ny = df_raw[\'VALUE_LT_STA6\'].shift(1)[1:]\nreg = RandomForestRegressor(n_estimators=100, random_state=42)\nreg = reg.fit(X, y)\n# Create the fingerprint model\nreg_fingerprint = RegressionModelFingerprint()\n# Fit the fingerprint model\n_ = reg_fingerprint.fit(\n     reg,\n     X,\n     num_values=20,)\n# Get linear non-linear effects and pairwise effects\nlinear_effect, non_linear_effect, pair_wise_effect = reg_fingerprint.get_effects()\n# Plot the results\nfig = reg_fingerprint.plot_effects(sort_by="non_lin")\nfig.show()'

In [5]:
class Dataset():#'forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
    def __init__(self, root_path=None, flag='train', size=None,
                 features='MS',target='target', scale=True, timeenc=0, freq='m'):

        # size [seq_len, label_len, pred_len]
        # info
        if size == None:
            print("size è none. Forse ci sono problemi")
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq
        self.root_path = root_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler =StandardScaler()
        df_raw = pd.read_csv('emanuele.csv')
        print(df_raw)
        border1s = [0, int(len(df_raw)*0.99)- self.seq_len, int(len(df_raw)*0.88)- self.seq_len]#[0, len(df_raw)-200- self.seq_len, len(df_raw)-100 - self.seq_len]#[0, int(len(df_raw)*0.95)- self.seq_len, int(len(df_raw)*0.98) - self.seq_len]
        border2s = [int(len(df_raw)*0.99) , int(len(df_raw)*1), int(len(df_raw)*9)]#[len(df_raw)-200 , len(df_raw) -100, int(len(df_raw))]#[int(len(df_raw)*0.95) , int(len(df_raw)*0.98), int(len(df_raw)*1)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]
        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]

        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
            self.scaler.fit(train_data['VALUE_LT_STA3'].values.reshape(-1, 1))
            #data=data.values
        else:
            data = df_data.values
        df_stamp = df_raw[['Date']][border1:border2]
        df_stamp['Date'] = pd.to_datetime(df_stamp.Date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.Date.apply(lambda row: row.month, 1)
            df_stamp['minute'] = df_stamp.Date.apply(lambda row: row.minute, 10)
            df_stamp['hour'] = df_stamp.Date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(labels=['Date'], axis=1).values
            print(df_stamp)
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['Date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)
        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]

        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]
        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

class Dataset_Pred(Dataset):
    def __init__(self, root_path=None, flag='train', size=None,
                 features='MS',
                 target='target', scale=True, timeenc=0, freq='m'):
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['pred']

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv('emanuele.csv')
        border1 = len(df_raw) - self.seq_len
        border2 = len(df_raw)

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            self.scaler.fit(df_data.values)
            data = self.scaler.transform(df_data.values)
            self.scaler.fit(df_data['VALUE_LT_STA3'].values.reshape(-1, 1))
            data=data.values
        else:
            data = df_data.values
        tmp_stamp = df_raw[['Date']][border1:border2]
        tmp_stamp['Date'] = pd.to_datetime(tmp_stamp.Date)
        pred_dates = pd.date_range(tmp_stamp.Date.values[-1], periods=self.pred_len + 1, freq=self.freq)
        df_stamp = pd.DataFrame(columns=['Date'])
        df_stamp.Date = list(tmp_stamp.Date.values) + list(pred_dates[1:])
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.Date.apply(lambda row: row.month, 1)
            df_stamp['minute'] = df_stamp.Date.apply(lambda row: row.minute, 10)
            df_stamp['hour'] = df_stamp.Date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(labels=['Date'], axis=1).values
            print(df_stamp)
            data_stamp = df_stamp.drop(labels=['Date'], axis=1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['Date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data[border1:border2]
        print(pred_dates)
        self.data_y = data[border1:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_begin + self.label_len]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]
        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [6]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x

class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='m'):
        super(TimeFeatureEmbedding, self).__init__()
        print("qui non dovrei entrare")
        freq_map = {'h': 4, 't': 5, 's': 6, 'm': 1, 'a': 1, 'w': 2, 'd': 3, 'B': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()

class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='m'):
        super(TemporalEmbedding, self).__init__()

        hour_size = 24
        weekday_size = 8
        day_size = 32
        minute_size = 61
        month_size = 13


        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding

        self.hour_embed = Embed(hour_size, d_model)

        self.minute_embed = Embed(minute_size, d_model)
        self.month_embed = Embed(month_size, d_model)




    def forward(self, x):
        x = x.long()
        hour_x = self.hour_embed(x[:, :, 2])
        minute_x = self.minute_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])
        return minute_x + month_x+hour_x


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='m', dropout=0.1):# questa va cambiata in base alla tipologia
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq)# if embed_type == 'timeF' else TimeFeatureEmbedding(
            #d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        x = self.value_embedding(x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)

class TriangularCausalMask():
    def __init__(self, B, L, device="cuda"):
        mask_shape = [B, 1, L, L]
        with torch.no_grad():
            self._mask = torch.triu(torch.ones(mask_shape, dtype=torch.bool), diagonal=1).to(device)

    @property
    def mask(self):
        return self._mask

class ProbMask():
    def __init__(self, B, H, L, index, scores, device="cuda"):
        _mask = torch.ones(L, scores.shape[-1], dtype=torch.bool).to(device).triu(1)
        _mask_ex = _mask[None, None, :].expand(B, H, L, scores.shape[-1])
        indicator = _mask_ex[torch.arange(B)[:, None, None],
                    torch.arange(H)[None, :, None],
                    index, :].to(device)
        self._mask = indicator.view(scores.shape).to(device)

    @property
    def mask(self):
        return self._mask
class DSAttention(nn.Module):
    '''De-stationary Attention'''
    def __init__(self, mask_flag=True, factor=5, scale=None, attention_dropout=0.1, output_attention=False):
        super(DSAttention, self).__init__()
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        scale = self.scale or 1. / sqrt(E)

        tau = 1.0 if tau is None else tau.unsqueeze(1).unsqueeze(1)  # B x 1 x 1 x 1
        delta = 0.0 if delta is None else delta.unsqueeze(1).unsqueeze(1)  # B x 1 x 1 x S

        # De-stationary Attention, rescaling pre-softmax score with learned de-stationary factors
        scores = torch.einsum("blhe,bshe->bhls", queries, keys) * tau + delta

        if self.mask_flag:
            if attn_mask is None:
                attn_mask = TriangularCausalMask(B, L, device=queries.device)

            scores.masked_fill_(attn_mask.mask, -np.inf)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        V = torch.einsum("bhls,bshd->blhd", A, values)

        if self.output_attention:
            return (V.contiguous(), A)
        else:
            return (V.contiguous(), None)

class AttentionLayer(nn.Module):
    def __init__(self, attention, d_model, n_heads, d_keys=None,
                 d_values=None):
        super(AttentionLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)
        d_values = d_values or (d_model // n_heads)

        self.inner_attention = attention
        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_model, d_keys * n_heads)
        self.value_projection = nn.Linear(d_model, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, _ = queries.shape
        _, S, _ = keys.shape
        H = self.n_heads

        queries = self.query_projection(queries).view(B, L, H, -1)
        keys = self.key_projection(keys).view(B, S, H, -1)
        values = self.value_projection(values).view(B, S, H, -1)

        out, attn = self.inner_attention(
            queries,
            keys,
            values,
            attn_mask,
            tau, delta
        )
        out = out.view(B, L, -1)

        return self.out_projection(out), attn
    
class EncoderLayer(nn.Module):
    def __init__(self, attention, d_model, d_ff=None, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        new_x, attn = self.attention(
            x, x, x,
            attn_mask=attn_mask,
            tau=tau, delta=delta
        )
        x = x + self.dropout(new_x)

        y = x = self.norm1(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))

        return self.norm2(x + y), attn


class Encoder(nn.Module):
    def __init__(self, attn_layers, conv_layers=None, norm_layer=None):
        super(Encoder, self).__init__()
        self.attn_layers = nn.ModuleList(attn_layers)
        self.conv_layers = nn.ModuleList(conv_layers) if conv_layers is not None else None
        self.norm = norm_layer

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        # x [B, L, D]
        attns = []
        if self.conv_layers is not None:
            for i, (attn_layer, conv_layer) in enumerate(zip(self.attn_layers, self.conv_layers)):
                delta = delta if i==0 else None
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                x = conv_layer(x)
                attns.append(attn)
            x, attn = self.attn_layers[-1](x, tau=tau, delta=None)
            attns.append(attn)
        else:
            for attn_layer in self.attn_layers:
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                attns.append(attn)

        if self.norm is not None:
            x = self.norm(x)

        return x, attns


class DecoderLayer(nn.Module):
    def __init__(self, self_attention, cross_attention, d_model, d_ff=None,
                 dropout=0.1, activation="relu"):
        super(DecoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu
 
    def forward(self, x, cross, x_mask=None, cross_mask=None, tau=None, delta=None):
        x = x + self.dropout(self.self_attention(
            x, x, x,
            attn_mask=x_mask,
            tau=tau, delta=None
        )[0])
        x = self.norm1(x)

        x = x + self.dropout(self.cross_attention(
            x, cross, cross,
            attn_mask=cross_mask,
            tau=tau, delta=delta
        )[0])

        y = x = self.norm2(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))

        return self.norm3(x + y)


class Decoder(nn.Module):
    def __init__(self, layers, norm_layer=None, projection=None):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList(layers)
        self.norm = norm_layer
        self.projection = projection

    def forward(self, x, cross, x_mask=None, cross_mask=None, tau=None, delta=None):
        for layer in self.layers:
            x = layer(x, cross, x_mask=x_mask, cross_mask=cross_mask, tau=tau, delta=delta)

        if self.norm is not None:
            x = self.norm(x)

        if self.projection is not None:
            x = self.projection(x)
        return x

class Projector(nn.Module):
    '''
    MLP to learn the De-stationary factors
    '''
    def __init__(self, enc_in, seq_len, hidden_dims, hidden_layers, output_dim, kernel_size=3):
        super(Projector, self).__init__()

        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.series_conv = nn.Conv1d(in_channels=seq_len, out_channels=1, kernel_size=kernel_size, padding=padding, padding_mode='circular', bias=False)

        layers = [nn.Linear(2 * enc_in, hidden_dims[0]), nn.ReLU()]
        for i in range(hidden_layers-1):
            layers += [nn.Linear(hidden_dims[i], hidden_dims[i+1]), nn.ReLU()]

        layers += [nn.Linear(hidden_dims[-1], output_dim, bias=False)]
        self.backbone = nn.Sequential(*layers)

    def forward(self, x, stats):
        # x:     B x S x E
        # stats: B x 1 x E
        # y:     B x O
        batch_size = x.shape[0]
        x = self.series_conv(x)          # B x 1 x E
        x = torch.cat([x, stats], dim=1) # B x 2 x E
        x = x.view(batch_size, -1) # B x 2E
        y = self.backbone(x)       # B x O
        return y


class Model(nn.Module):
    """
    Non-stationary Transformer
    """
    def __init__(self, pred_len,seq_len,label_len,output_attention='store_true',enc_in=1,dec_in=1,d_model=512,embed='timeF',
                 freq='m',dropout=0.05,factor=1,e_layers=1,n_heads=16,d_ff=2048,p_hidden_dims=[128, 128],
                 p_hidden_layers=2,activation='gelu',d_layers=1,c_out=1):
        super(Model, self).__init__()
        self.pred_len = pred_len
        self.seq_len = seq_len
        self.label_len = label_len
        self.output_attention = output_attention
        self.enc_in=enc_in
        self.dec_in=dec_in
        self.d_model=d_model
        self.embed=embed
        self.freq=freq
        self.dropout=dropout
        self.factor=factor
        self.e_layers=e_layers
        self.n_heads=n_heads
        self.d_ff=d_ff
        self.p_hidden_dims=p_hidden_dims
        self.p_hidden_layers=p_hidden_layers
        self.activation=activation
        self.d_layers=d_layers
        self.c_out=c_out


        # Embedding
        self.enc_embedding = DataEmbedding(enc_in, d_model, embed, freq,
                                           dropout)
        self.dec_embedding = DataEmbedding(dec_in, d_model, embed, freq,
                                           dropout)
        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        DSAttention(False, factor, attention_dropout=dropout,
                                      output_attention=output_attention), d_model, n_heads),
                    d_model,
                    d_ff,
                    dropout=dropout,
                    activation=activation
                ) for l in range(e_layers)
            ],
            norm_layer=torch.nn.LayerNorm(d_model)
        )
        # Decoder
        self.decoder = Decoder(
            [
                DecoderLayer(
                    AttentionLayer(
                        DSAttention(True, factor, attention_dropout=dropout, output_attention=False),
                        d_model, n_heads),
                    AttentionLayer(
                        DSAttention(False, factor, attention_dropout=dropout, output_attention=False),
                        d_model, n_heads),
                    d_model,
                    d_ff,
                    dropout=dropout,
                    activation=activation,
                )
                for l in range(self.d_layers)
            ],
            norm_layer=torch.nn.LayerNorm(d_model),
            projection=nn.Linear(d_model, c_out, bias=True)
        )

        self.tau_learner   = Projector(enc_in=enc_in, seq_len=seq_len, hidden_dims=p_hidden_dims, hidden_layers=p_hidden_layers, output_dim=1)
        self.delta_learner = Projector(enc_in=enc_in, seq_len=seq_len, hidden_dims=p_hidden_dims, hidden_layers=p_hidden_layers, output_dim=seq_len)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec,
                enc_self_mask=None, dec_self_mask=None, dec_enc_mask=None):

        x_raw = x_enc.clone().detach()
        # Normalization
        mean_enc = x_enc.mean(1, keepdim=True).detach() # B x 1 x E
        x_enc = x_enc - mean_enc
        std_enc = torch.sqrt(torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5).detach() # B x 1 x E
        x_enc = x_enc / std_enc
        x_dec_new = torch.cat([x_enc[:, -self.label_len: , :], torch.zeros_like(x_dec[:, -self.pred_len:, :])], dim=1).to(x_enc.device).clone()

        tau = self.tau_learner(x_raw, std_enc).exp()     # B x S x E, B x 1 x E -> B x 1, positive scalar
        delta = self.delta_learner(x_raw, mean_enc)      # B x S x E, B x 1 x E -> B x S

        # Model Inference
        enc_out = self.enc_embedding(x_enc, x_mark_enc)
        enc_out, attns = self.encoder(enc_out, attn_mask=enc_self_mask, tau=tau, delta=delta)

        dec_out = self.dec_embedding(x_dec_new, x_mark_dec)
        dec_out = self.decoder(dec_out, enc_out, x_mask=dec_self_mask, cross_mask=dec_enc_mask, tau=tau, delta=delta)

        # De-normalization
        dec_out = dec_out * std_enc + mean_enc
        output=dec_out[:, -self.pred_len:, :]
        if self.output_attention:
            return output, attns
        else:
            return output # [B, L, D]



In [7]:
class DNNModel(object):
    def __init__(self,batch_size, epochs_early_stopping=20):
        self.embed='fixed'
        self.batch_size=batch_size
        self.freq='10m'
        self.num_workers=0
        self.pred_len=6*1
        self.label_len=6
        self.seq_len=1000
        self.model = self._build_model()
        self.use_amp=False
        self.train_epochs=1000
        self.features='MS'
        self.output_attention=True
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.checkpoints='./checkpoints/'
        self.lr=0.001


    def data_provider(self,flag):
        data_dict = {
            'ETTh1': Dataset
        }
        Data = data_dict['ETTh1']
        timeenc = 0 #if self.embed == 'timeF' else 1
        if flag == 'test':
            shuffle_flag = False
            drop_last = False
            batch_size = 1
            freq = self.freq
        elif flag == 'pred':
            shuffle_flag = False
            drop_last = False
            batch_size =  1
            freq = self.freq
            Data = Dataset_Pred
        elif flag == 'val':
            shuffle_flag = False
            drop_last = False
            batch_size =  1
            freq = self.freq
        else:
            shuffle_flag=True
            drop_last = True
            batch_size =self.batch_size
            freq = self.freq

        data_set = Data(
            flag=flag,
            size=[self.seq_len, self.label_len,self.pred_len],
            freq=self.freq
        )
        print(flag, len(data_set))
        data_loader = DataLoader(
            data_set,
            batch_size=self.batch_size,
            shuffle=shuffle_flag,
            num_workers=self.num_workers,
            drop_last=drop_last)
        return data_set, data_loader

    def _build_model(self):
        model = Model(pred_len=self.pred_len,seq_len=self.seq_len,label_len=self.label_len,output_attention=True).to("cuda")
        return model
    
    def _select_optimizer(self):
        model_optim = optim.Adam(self.model.parameters(), lr=self.lr)#,weight_decay=0.0005
        return model_optim

    def _select_criterion(self):
        criterion = nn.HuberLoss(reduction='mean', delta=1.0)#nn.MSELoss()#nn.CrossEntropyLoss()#nn.BCELoss()#nn.BCEWithLogitsLoss() #nn.NLLLoss()#nn.HuberLoss(reduction='mean', delta=1.0)#nn.MSELoss()#nn.BCELoss()#nn.L1Loss()#nn.HuberLoss(reduction='mean', delta=1.0)#nn.MSELoss()#nn.HuberLoss(reduction='mean', delta=1.0)#nn.MSELoss()
        return criterion

    def _get_data(self, flag):
        data_set, data_loader = self.data_provider(flag)
        return data_set, data_loader

    def vali(self, vali_data, vali_loader, criterion):
        total_loss = []
        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float()

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.pred_len:, :]).float()#qui penso di dover mettere l'output
                dec_inp = torch.cat([batch_y[:, :self.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                else:
                    if self.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1 if (self.features == 'MS' or self.features =='S')  else 0
                outputs = outputs[:, -self.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.pred_len:, f_dim:].to(self.device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)
                total_loss.append(loss)

        total_loss = np.average(total_loss)
        self.model.train()
        return total_loss


    def train(self,setting):
        train_data, train_loader = self._get_data(flag='train')
        vali_data, vali_loader = self._get_data(flag='val')
        #test_data, test_loader = self._get_data(flag='test')

        path = os.path.join(self.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()

        train_steps = len(train_loader)
        early_stopping = EarlyStopping(patience=100, verbose=True)

        model_optim = self._select_optimizer()
        criterion = self._select_criterion()
        scheduler = ReduceLROnPlateau(model_optim, mode='min', patience=3, factor=0.5, verbose=False)
        if self.use_amp:
            scaler = torch.cuda.amp.GradScaler()

        for epoch in range(self.train_epochs):
            iter_count = 0
            train_loss = []
            self.model.train()
            epoch_time = time.time()
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                model_optim.zero_grad()
                batch_x = batch_x.float().to(self.device)

                batch_y = batch_y.float().to(self.device)
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.label_len, :], dec_inp], dim=1).float().to(self.device)

                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                        f_dim = -1 if (self.features == 'MS' or self.features =='S')  else 0
                        outputs = outputs[:, -self.pred_len:, f_dim:]
                        batch_y = batch_y[:, -self.pred_len:, f_dim:].to(self.device)
                        loss = criterion(outputs, batch_y)
                        train_loss.append(loss.item())
                else:
                    if self.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                    f_dim = -1 if(self.features == 'MS' or self.features =='S')  else 0
                    outputs = outputs[:, -self.pred_len:, f_dim:]
                    #print(outputs)
                    batch_y = batch_y[:, -self.pred_len:, f_dim:].to(self.device)
                    #print(batch_y )
                    loss = criterion(outputs, batch_y)
                    train_loss.append(loss.item())

                if (i + 1) % 100 == 0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.train_epochs - epoch) * train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()

                if self.use_amp:
                    scaler.scale(loss).backward()
                    scaler.step(model_optim)
                    scaler.update()
                else:
                    loss.backward()
                    model_optim.step()

            print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss)
            vali_loss= self.vali(vali_data, vali_loader, criterion)

            scheduler.step(vali_loss)
            #test_loss = self.vali(test_data, test_loader, criterion)

            #print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
            #    epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f}".format(
                epoch + 1, train_steps, train_loss, vali_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

        best_model_path = path + '/' + 'checkpoint.pth'
        self.model.load_state_dict(torch.load(best_model_path))
        return self.model

    def vali_test(self, setting, test=True):
        test_data, test_loader = self._get_data(flag='val')
        if test:
            print('loading model')
            self.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))

        preds = []
        trues = []
        folder_path = './test_results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                else:
                    if self.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]

                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1 if (self.features == 'MS' or self.features =='S') else 0
                outputs = outputs[:, -self.pred_len:, f_dim:]
                #outputs =torch.argmax(outputs, dim=1)
                batch_y = batch_y[:, -self.pred_len:, f_dim:].to(self.device)
                outputs = outputs.detach().cpu().numpy()
                batch_y = batch_y.detach().cpu().numpy()
                pred = outputs  # outputs.detach().cpu().numpy()  # .squeeze()
                true = batch_y  # batch_y.detach().cpu().numpy()  # .squeeze()
                preds.append(pred)
                trues.append(true)


        preds = np.array(preds)
        trues = np.array(trues)
        print('test shape:', preds.shape, trues.shape)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
        print('test shape:', preds.shape, trues.shape)

        # result save
        folder_path = './results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        #mae, mse, rmse, mape, mspe = metric(preds, trues)
        #print('mse:{}, mae:{}'.format(mse, mae))
        #f = open("result.txt", 'a')
        #f.write(setting + "  \n")
        #f.write('mse:{}, mae:{}'.format(mse, mae))
        #f.write('\n')
        #f.write('\n')
        #f.close()

        #np.save(folder_path + 'metrics.npy', np.array([mae, mse, rmse, mape, mspe]))
        #np.save(folder_path + 'pred.npy', preds)
        #np.save(folder_path + 'true.npy', trues)

        return preds,trues




    def predict(self, setting, load=True):
        pred_data, pred_loader = self._get_data(flag='pred')

        if load:
            path = os.path.join(self.checkpoints, setting)
            best_model_path = path + '/' + 'checkpoint.pth'
            self.model.load_state_dict(torch.load(best_model_path))

        preds = []
        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(pred_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float()
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros([batch_y.shape[0], self.pred_len, batch_y.shape[2]]).float()
                dec_inp = torch.cat([batch_y[:, :self.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        if self.output_attention:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                else:
                    if self.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                pred = outputs.detach().cpu().numpy()  # .squeeze()
                preds.append(pred)
                #print(preds)
        preds = np.array(preds)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])

        # result save
        #folder_path = './results/' + setting + '/'
        #if not os.path.exists(folder_path):
        #    os.makedirs(folder_path)

        #np.save(folder_path + 'real_prediction.npy', preds)

        return preds

In [8]:
def _build_space():
    space = {
        'batch_size': hp.quniform('batch_size',20,21,1),
        }
    return space



def _hyperopt_objective(hyperparameters, trials, trials_file_path, max_evals):

    #print(hyperparameters)


    pc.dump(trials, open(trials_file_path, "wb"))
    setting = '{}'.format('EMANUELE')

    forecaster = DNNModel(
        batch_size=int(hyperparameters['batch_size'])
                     )

    forecaster.train(setting).to("cuda")
    Yp_mean ,Y_test= forecaster.test(setting,test=True)


    #Y_test = Dataset().inverse_transform(Y_test.reshape(-1, Y_test.shape[-1])).flatten()
    #Yp_mean = Dataset().inverse_transform(Yp_mean.reshape(-1, Yp_mean.shape[-1])).flatten()
    #print("Yp_mean",Yp_mean)
    #print("Y_test",Y_test)

    mae_validation = np.mean(MAE(Yp_mean, Y_test))
    smape_validation = np.mean(sMAPE(Y_test, Yp_mean))*100
    return_values = {'loss': mae_validation, 'MAE Val': mae_validation,'sMAPE Val': smape_validation,
                     'status': STATUS_OK}

    if trials.losses()[0] is not None:
        MAEVal = trials.best_trial['result']['MAE Val']
        sMAPEVal = trials.best_trial['result']['sMAPE Val']

        print('\n\nTested {}/{} iterations.'.format(len(trials.losses()) - 1,
              max_evals))
        print('Best MAE - Validation Dataset')
        print("  MAE: {:.1f} | sMAPE: {:.2f} %".format(MAEVal, sMAPEVal))
    return return_values

def hyperparameter_optimizer(path_hyperparameters_folder=os.path.join('.', 'experimental_files'),
                             new_hyperopt=1, max_evals=1500,
                             experiment_id=None):

    if not os.path.exists(path_hyperparameters_folder):
        os.makedirs(path_hyperparameters_folder)
    if experiment_id is None:
        experiment_id = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
    else:
        experiment_id = experiment_id
    trials_file_name = 'DNN_hyperparameters_SP500_15m_try3'
    trials_file_path = os.path.join(path_hyperparameters_folder, trials_file_name)
    if new_hyperopt:
        trials = Trials()
    else:
        trials = pc.load(open(trials_file_path, "rb"))
    # n_exogenous_inputs = len(df.columns) - 1
    space = _build_space()

    fmin_objective = partial(_hyperopt_objective, trials=trials, trials_file_path=trials_file_path,
                             max_evals=max_evals)
    fmin(fmin_objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials, verbose=False)

In [9]:
new_hyperopt = 1#0.0065117
experiment_id = 1
max_evals = 1
path_hyperparameters_folder = "./experimental_files/"
hyperparameter_optimizer(path_hyperparameters_folder=path_hyperparameters_folder,
                         new_hyperopt=new_hyperopt, max_evals=max_evals,
                         experiment_id=experiment_id)#0.0814 di mae nell'articolo#0.0145(0.0037) MSE'''

                       Date  VALUE_LT_STA3
0       2021-06-15 09:30:00           24.6
1       2021-06-15 09:40:00           24.8
2       2021-06-15 09:50:00           25.0
3       2021-06-15 10:00:00           25.1
4       2021-06-15 10:10:00           25.4
...                     ...            ...
109948  2023-07-18 23:20:00           19.3
109949  2023-07-18 23:30:00           19.4
109950  2023-07-18 23:40:00           19.6
109951  2023-07-18 23:50:00           19.8
109952  2023-07-19 00:00:00           19.8

[109953 rows x 2 columns]
                      Date  month  minute  hour
0      2021-06-15 09:30:00      6      30     9
1      2021-06-15 09:40:00      6      40     9
2      2021-06-15 09:50:00      6      50     9
3      2021-06-15 10:00:00      6       0    10
4      2021-06-15 10:10:00      6      10    10
...                    ...    ...     ...   ...
108848 2023-07-11 08:00:00      7       0     8
108849 2023-07-11 08:10:00      7      10     8
108850 2023-07-11 08:20:0

KeyboardInterrupt: ignored

In [ ]:
def MAE(pred, true):
    return np.mean(np.abs(pred - true))


def MSE(pred, true):
    return np.mean((pred - true) ** 2)


def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))


def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))


def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))

In [ ]:
label_len=200
seq_len=450
pred_len=6
forecaster = DNNModel(batch_size=1)
setting = '{}'.format('EMANUELE')
Yp_mean ,Y_test= forecaster.vali_test(setting, test=True)
data=Dataset(flag='val',size=[seq_len,label_len,pred_len],freq='10m')

mae_validation = np.mean(MAE(Yp_mean, Y_test))
mse_validation = np.mean(MSE(Yp_mean, Y_test))
smape_validation = np.mean(MAPE(Y_test, Yp_mean))*100
print("MAE: {:.3f} | MAPE: {:.6f} % | MSE: {:.6f}".format(mae_validation, smape_validation,mse_validation))
Y_test = data.inverse_transform(Y_test.reshape(-1, 1)).flatten()
Yp_mean = data.inverse_transform(Yp_mean.reshape(-1, 1)).flatten()
mae_validation = np.mean(MAE(Yp_mean, Y_test))
mse_validation = np.mean(MSE(Yp_mean, Y_test))
smape_validation = np.mean(MAPE(Y_test, Yp_mean))*100
time = range(len(Y_test))
print("MAE: {:.3f} | MAPE: {:.4f} % | MSE: {:.4f}".format(mae_validation, smape_validation,mse_validation))
plt.plot(time, Y_test.flatten(), color='blue', label='Prezzi Veri')
plt.plot(time, Yp_mean.flatten(), color='red', label='Previsioni')
plt.xlabel('Tempo')
plt.ylabel('Temperaure')
plt.legend()
plt.show()
differenza=np.abs(Yp_mean-Y_test).flatten()
time = range(len(differenza))
plt.plot(time, differenza, label='previsione-vero')
plt.xlabel('Time')
plt.ylabel('previsione-vero')
plt.title('previsione-vero')
plt.show()
difference=(Yp_mean-Y_test).flatten()
plt.figure(figsize=(10,5),dpi=100)
plt.hist(difference,bins=100)
plt.show()
print("media istogramma",np.mean(difference))

In [ ]:
label_len=6
seq_len=1000
pred_len=6
forecaster = DNNModel(batch_size=1)
setting = '{}'.format('EMANUELE')
Yp_mean ,Y_test= forecaster.vali_test(setting, test=True)
Yp_mean =Yp_mean[:,-1,0]
Y_test=Y_test[:,-1,0]
data=Dataset(flag='val',size=[seq_len,label_len,pred_len],freq='10m')
mae_validation = np.mean(MAE(Yp_mean, Y_test))
mse_validation = np.mean(MSE(Yp_mean, Y_test))
smape_validation = np.mean(MAPE(Y_test, Yp_mean))*100
print("MAE: {:.3f} | MAPE: {:.6f} % | MSE: {:.6f}".format(mae_validation, smape_validation,mse_validation))
Y_test = data.inverse_transform(Y_test.reshape(-1, 1)).flatten()
Yp_mean = data.inverse_transform(Yp_mean.reshape(-1, 1)).flatten()
mae_validation = np.mean(MAE(Yp_mean, Y_test))
mse_validation = np.mean(MSE(Yp_mean, Y_test))
smape_validation = np.mean(MAPE(Y_test, Yp_mean))*100
time = range(len(Y_test))
print("MAE: {:.3f} | MAPE: {:.4f} % | MSE: {:.4f}".format(mae_validation, smape_validation,mse_validation))
plt.plot(time, Y_test.flatten(), color='blue', label='Prezzi Veri')
plt.plot(time, Yp_mean.flatten(), color='red', label='Previsioni')
plt.xlabel('Tempo')
plt.ylabel('Temperaure')
plt.legend()
plt.show()
differenza=np.abs(Yp_mean-Y_test).flatten()
time = range(len(differenza))
plt.plot(time, differenza, label='previsione-vero')
plt.xlabel('Time')
plt.ylabel('previsione-vero')
plt.title('previsione-vero')
plt.show()
difference=(Yp_mean-Y_test).flatten()
plt.figure(figsize=(10,5),dpi=100)
plt.hist(difference,bins=100)
plt.show()
print("media istogramma",np.mean(difference))

In [ ]:
print(Yp_mean[:,-1,0].shape)